In [1]:
import numpy as np
import pandas as pd
from scipy import stats
from scipy import signal
import pprint
from sklearn import preprocessing
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import datetime
import time
import seaborn as sns
import matplotlib.cm as cm
import statsmodels as sm
import pickle

In [2]:
# weight_type = 'ff100'
weight_type = 'ons'

In [3]:
with open('data/GB/region_names/iso_names.txt', mode='r') as f:
    regions = f.read().splitlines()
    
name_to_omit = ['London, City of', 'Isles of Scily']

regions = [r.replace(' ', '_') for r in regions if r not in name_to_omit]
metrics = ['newCasesBySpecimenDate', 'newDeaths28DaysByDeathDate']

dfs = {}
for r in regions:
    for m in metrics:
        if m == metrics[0]:
            clinical = pd.read_csv('data/GB/regional/clinical/{}/{}.csv'.format(m, r), keep_default_na=False, na_values=[''])
            clinical = clinical.dropna(how='any').drop(['Unnamed: 0', 'areaType'], axis=1)
            clinical['date'] = pd.to_datetime(clinical['date'])
            clinical = clinical.set_index('date')
        else:
            try:
                tmp = pd.read_csv('data/GB/regional/clinical/{}/{}.csv'.format(m, r), keep_default_na=False, na_values=[''])
            except Exception as exc:
                break
            tmp['date'] = pd.to_datetime(tmp['date'])
            tmp = tmp.set_index('date')
#             clinical = pd.concat([clinical, tmp[m]], axis=1)
            clinical = pd.merge(clinical, tmp[m], left_index=True, right_index=True, how='outer')

    idx = clinical.columns.get_loc('nhsName')+1
    if list(clinical.columns.values[idx:]) != metrics:
        continue
    searches = pd.read_csv('data/GB/regional/searches/{}.csv'.format(r), keep_default_na=False, na_values=[''])
    searches = searches.drop(['country_region_code', 'country_region', 'sub_region_1', 'sub_region_1_code', 'sub_region_2', 'sub_region_2_code', 'place_id'], axis=1)
    searches['date'] = pd.to_datetime(searches['date'])
#         if r == 'Hackney':
#             tmp = pd.read_csv('data/GB/regional/searches/{}.csv'.format('London,_City_of'), keep_default_na=False, na_values=[''])
#             tmp = tmp.drop(['country_region_code', 'country_region', 'sub_region_1', 'sub_region_1_code', 'sub_region_2', 'sub_region_2_code', 'place_id'], axis=1)
#             tmp['date'] = pd.to_datetime(tmp['date'].astype(str))
#             idx = searches.columns.get_loc('date')+1
#             print(searches.iloc[:, idx:])
#             searches = pd.concat([searches.iloc[:, :idx], searches.iloc[:, idx:].add(tmp.iloc[:, idx:], fill_value=0.0)])
    df = pd.merge(clinical, searches, on='date', how='outer').iloc[::-1].sort_values(by=['date'])
    df = df.set_index('date')
    dfs[r.replace('_', ' ')] = df
print(len(dfs))
dfs['Aberdeen City']

189


,isoCode,areaCode,parentCode,isoName,googleName,nhsName,newCasesBySpecimenDate,newDeaths28DaysByDeathDate,symptom:Abdominal obesity,symptom:Abdominal pain,...,symptom:Wart,symptom:Water retention,symptom:Weakness,symptom:Weight gain,symptom:Wheeze,symptom:Xeroderma,symptom:Xerostomia,symptom:Yawn,symptom:hyperhidrosis,symptom:pancreatitis
date,,,,,,,,,,,,,,,,,,,,,
2019-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.19,5.14,...,1.28,NaN,NaN,2.64,NaN,2.85,NaN,NaN,NaN,NaN
2019-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.66,5.02,...,1.59,NaN,0.95,3.77,NaN,3.31,NaN,NaN,NaN,NaN
2019-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.27,5.70,...,1.55,NaN,0.79,3.10,NaN,3.24,0.62,NaN,0.7,NaN
2019-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.79,4.42,...,1.29,NaN,NaN,3.04,NaN,2.93,NaN,NaN,NaN,NaN
2019-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.95,4.86,...,1.29,NaN,NaN,3.38,NaN,3.08,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-07-13,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,81.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-07-14,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,59.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-07-15,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,54.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
for i, key in enumerate(dfs.keys()):
    dfs[key] = dfs[key].loc[:'2022-05-19']
dfs['Aberdeen City']

,isoCode,areaCode,parentCode,isoName,googleName,nhsName,newCasesBySpecimenDate,newDeaths28DaysByDeathDate,symptom:Abdominal obesity,symptom:Abdominal pain,...,symptom:Wart,symptom:Water retention,symptom:Weakness,symptom:Weight gain,symptom:Wheeze,symptom:Xeroderma,symptom:Xerostomia,symptom:Yawn,symptom:hyperhidrosis,symptom:pancreatitis
date,,,,,,,,,,,,,,,,,,,,,
2019-01-01,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.19,5.14,...,1.28,NaN,NaN,2.64,NaN,2.85,NaN,NaN,NaN,NaN
2019-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.66,5.02,...,1.59,NaN,0.95,3.77,NaN,3.31,NaN,NaN,NaN,NaN
2019-01-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.27,5.70,...,1.55,NaN,0.79,3.10,NaN,3.24,0.62,NaN,0.70,NaN
2019-01-04,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.79,4.42,...,1.29,NaN,NaN,3.04,NaN,2.93,NaN,NaN,NaN,NaN
2019-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.95,4.86,...,1.29,NaN,NaN,3.38,NaN,3.08,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-15,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,42.0,0.0,4.78,7.79,...,2.28,NaN,NaN,4.16,NaN,4.13,NaN,NaN,1.12,NaN
2022-05-16,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,57.0,0.0,3.27,7.90,...,1.33,NaN,NaN,3.80,NaN,3.80,NaN,NaN,1.33,NaN
2022-05-17,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,44.0,0.0,3.38,6.82,...,2.06,NaN,NaN,4.84,NaN,3.56,NaN,NaN,0.96,NaN


In [6]:
columns = dfs['Aberdeen City'].columns[:dfs['Aberdeen City'].columns.get_loc('nhsName')+1].values
for key in dfs.keys():
    for c in columns:
        dfs[key][c] = dfs[key][c].fillna(dfs[key][c].iloc[500])
dfs['Aberdeen City']

/var/folders/16/p0_pv93912d7zzx29sykqy800000gn/T/ipykernel_1470/2140708226.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs[key][c] = dfs[key][c].fillna(dfs[key][c].iloc[500])


,isoCode,areaCode,parentCode,isoName,googleName,nhsName,newCasesBySpecimenDate,newDeaths28DaysByDeathDate,symptom:Abdominal obesity,symptom:Abdominal pain,...,symptom:Wart,symptom:Water retention,symptom:Weakness,symptom:Weight gain,symptom:Wheeze,symptom:Xeroderma,symptom:Xerostomia,symptom:Yawn,symptom:hyperhidrosis,symptom:pancreatitis
date,,,,,,,,,,,,,,,,,,,,,
2019-01-01,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,NaN,NaN,2.19,5.14,...,1.28,NaN,NaN,2.64,NaN,2.85,NaN,NaN,NaN,NaN
2019-01-02,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,NaN,NaN,2.66,5.02,...,1.59,NaN,0.95,3.77,NaN,3.31,NaN,NaN,NaN,NaN
2019-01-03,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,NaN,NaN,3.27,5.70,...,1.55,NaN,0.79,3.10,NaN,3.24,0.62,NaN,0.70,NaN
2019-01-04,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,NaN,NaN,2.79,4.42,...,1.29,NaN,NaN,3.04,NaN,2.93,NaN,NaN,NaN,NaN
2019-01-05,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,NaN,NaN,2.95,4.86,...,1.29,NaN,NaN,3.38,NaN,3.08,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-15,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,42.0,0.0,4.78,7.79,...,2.28,NaN,NaN,4.16,NaN,4.13,NaN,NaN,1.12,NaN
2022-05-16,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,57.0,0.0,3.27,7.90,...,1.33,NaN,NaN,3.80,NaN,3.80,NaN,NaN,1.33,NaN
2022-05-17,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,44.0,0.0,3.38,6.82,...,2.06,NaN,NaN,4.84,NaN,3.56,NaN,NaN,0.96,NaN


In [7]:
if weight_type == 'ff100':
    symptoms = ["Anosmia", "Arthralgia", "Clouding of consciousness", "Cough", "Diarrhea", "Fatigue", "Fever", "Headache", "Myalgia", 
            "Nausea", "Nosebleed", "Rhinitis", "Shortness of breath", "Skin rash", "Sore throat", "Vomiting",] 
    
elif weight_type == 'ons':
    # symptoms = ["Abdominal pain",  "Ageusia", "Anosmia", "Chills", "Cluster headache", "Cough", "Diarrhea", "Dysgeusia", 
    #             "Fatigue", "Fever", "Headache", "Low-grade fever", "Myalgia", "Nausea", "Rhinitis", "Shallow breathing", "Shortness of breath", 
    #             "Sore throat", "Throat irritation", "Vomiting", ] 
#     symptoms = ["Abdominal pain",  "Ageusia", "Anosmia", "Chills", "Cluster headache", "Cough", "Diarrhea", "Dysgeusia", 
#                 "Fatigue", "Fever", "Headache", "Low-grade fever", "Myalgia", "Nausea", "Rhinitis", "Shallow breathing", "Shortness of breath", 
#                 "Sore throat", "Throat irritation", "Vomiting", "Weakness", ] 
#     symptoms = ["Ageusia", "Anosmia", "Cough", "Diarrhea", "Fatigue", "Fever", "Headache",  "Myalgia", "Nausea", "Rhinitis", 
#                 "Shortness of breath", "Sore throat", "Vomiting", ] 
#     symptoms = ["Ageusia", "Anosmia", "Cough", "Diarrhea", "Fatigue", "Fever", "Headache",  "Myalgia", "Nausea", 
#                 "Shortness of breath", "Sore throat", "Vomiting", ] 
    symptoms = ["Abdominal pain",  "Ageusia", "Anosmia", "Cough", "Diarrhea", "Fatigue", "Fever", "Headache", "Myalgia", "Nausea", 
                "Shortness of breath", "Sore throat", "Vomiting", "Weakness", ] 
#     symptoms = ["Ageusia", "Anosmia", "Cough", "Diarrhea", "Fatigue", "Fever", "Headache",  "Myalgia", "Nausea", 
#                 "Shortness of breath", "Sore throat", ] 
symptoms = ['symptom:' + s for s in symptoms]

In [8]:
column = df.columns.get_loc(metrics[-1])+1
for key in dfs.keys():
    dfs[key] = pd.concat([dfs[key].iloc[:, :column], dfs[key][symptoms]], axis=1)
dfs['Aberdeen City']

,isoCode,areaCode,parentCode,isoName,googleName,nhsName,newCasesBySpecimenDate,newDeaths28DaysByDeathDate,symptom:Abdominal pain,symptom:Ageusia,...,symptom:Diarrhea,symptom:Fatigue,symptom:Fever,symptom:Headache,symptom:Myalgia,symptom:Nausea,symptom:Shortness of breath,symptom:Sore throat,symptom:Vomiting,symptom:Weakness
date,,,,,,,,,,,,,,,,,,,,,
2019-01-01,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,NaN,NaN,5.14,NaN,...,4.83,4.03,3.47,4.31,NaN,1.77,1.01,2.71,3.72,NaN
2019-01-02,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,NaN,NaN,5.02,NaN,...,3.80,5.82,3.37,3.77,NaN,2.11,0.77,3.00,3.31,0.95
2019-01-03,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,NaN,NaN,5.70,NaN,...,4.54,6.32,4.12,4.43,0.59,2.00,NaN,2.76,3.92,0.79
2019-01-04,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,NaN,NaN,4.42,NaN,...,4.31,6.21,4.14,4.51,NaN,2.30,0.78,3.71,3.65,NaN
2019-01-05,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,NaN,NaN,4.86,NaN,...,4.49,6.06,3.51,4.92,0.68,2.19,NaN,1.66,2.65,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-15,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,42.0,0.0,7.79,NaN,...,7.42,8.51,5.87,6.95,0.94,2.50,0.94,4.42,5.36,NaN
2022-05-16,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,57.0,0.0,7.90,NaN,...,7.87,8.75,4.83,5.16,0.62,3.06,1.33,3.33,5.25,NaN
2022-05-17,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,44.0,0.0,6.82,NaN,...,6.37,10.10,4.78,5.71,0.96,2.99,1.52,4.21,4.60,NaN


In [16]:
idx = clinical.columns.get_loc(metrics[-1])+1
nans = []
for key in dfs.keys():  
    nans.append((len(dfs[key]) - np.array([dfs[key][c].isnull().sum() for c in dfs[key].columns[idx:]]))/len(dfs[key]))
search_stats = pd.DataFrame(nans, index=dfs.keys(), columns=symptoms)
search_stats.describe().T

,count,mean,std,min,25%,50%,75%,max
symptom:Abdominal pain,189.0,0.985926,0.108826,0.076113,1.000000,1.000000,1.000000,1.000000
symptom:Ageusia,189.0,0.043738,0.123344,0.000000,0.000000,0.000000,0.000000,0.592713
symptom:Anosmia,189.0,0.088829,0.187300,0.000000,0.000000,0.000000,0.000000,0.702834
symptom:Cough,189.0,0.985408,0.107109,0.127126,1.000000,1.000000,1.000000,1.000000
symptom:Diarrhea,189.0,0.984320,0.116659,0.058300,1.000000,1.000000,1.000000,1.000000
symptom:Fatigue,189.0,0.987978,0.094250,0.195951,1.000000,1.000000,1.000000,1.000000
symptom:Fever,189.0,0.983493,0.116319,0.072065,1.000000,1.000000,1.000000,1.000000
symptom:Headache,189.0,0.984495,0.115964,0.069636,1.000000,1.000000,1.000000,1.000000
symptom:Myalgia,189.0,0.678534,0.406435,0.000000,0.417814,0.924696,0.995951,1.000000
symptom:Nausea,189.0,0.966647,0.150058,0.006478,0.999190,1.000000,1.000000,1.000000


In [10]:
nulls = {}
names= {}
for c in dfs[key].columns[idx:]:
    cnt = 0
    names[c] = []
    for key in dfs.keys():
        if dfs[key][c].isnull().sum() == len(dfs[key]):
            cnt += 1
        else:
            names[c].append(key) 
    nulls[c] = cnt
print(len(dfs))
print(nulls)
print(names['symptom:Anosmia'])

189
{'symptom:Abdominal pain': 0, 'symptom:Ageusia': 163, 'symptom:Anosmia': 151, 'symptom:Cough': 0, 'symptom:Diarrhea': 0, 'symptom:Fatigue': 0, 'symptom:Fever': 0, 'symptom:Headache': 0, 'symptom:Myalgia': 45, 'symptom:Nausea': 0, 'symptom:Shortness of breath': 12, 'symptom:Sore throat': 0, 'symptom:Vomiting': 0, 'symptom:Weakness': 60}
['Barnet', 'Birmingham', 'Bournemouth, Christchurch and Poole', 'Bradford', 'Bromley', 'Cambridgeshire', 'Derbyshire', 'Devon', 'Ealing', 'Essex', 'Glasgow City', 'Gloucestershire', 'Hampshire', 'Hertfordshire', 'Kent', 'Kirklees', 'Lambeth', 'Lancashire', 'Leeds', 'Leicestershire', 'Lincolnshire', 'Liverpool', 'Manchester', 'Newham', 'Norfolk', 'North Yorkshire', 'Northamptonshire', 'Nottinghamshire', 'Oxfordshire', 'Sheffield', 'Southwark', 'Suffolk', 'Surrey', 'Tower Hamlets', 'Wandsworth', 'Warwickshire', 'West Sussex', 'Westminster']


In [887]:
column = dfs['Aberdeen City'].columns.get_loc('newCasesBySpecimenDate')
for key in dfs.keys():
    dfs[key].iloc[:, column:]= dfs[key].iloc[:, column:].fillna(0)
dfs['Aberdeen City']

,isoCode,areaCode,parentCode,isoName,googleName,nhsName,newCasesBySpecimenDate,newDeaths28DaysByDeathDate,symptom:Abdominal pain,symptom:Ageusia,...,symptom:Diarrhea,symptom:Fatigue,symptom:Fever,symptom:Headache,symptom:Myalgia,symptom:Nausea,symptom:Shortness of breath,symptom:Sore throat,symptom:Vomiting,symptom:Weakness
date,,,,,,,,,,,,,,,,,,,,,
2019-01-01,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,0.0,0.0,5.14,0.0,...,4.83,4.03,3.47,4.31,0.00,1.77,1.01,2.71,3.72,0.00
2019-01-02,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,0.0,0.0,5.02,0.0,...,3.80,5.82,3.37,3.77,0.00,2.11,0.77,3.00,3.31,0.95
2019-01-03,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,0.0,0.0,5.70,0.0,...,4.54,6.32,4.12,4.43,0.59,2.00,0.00,2.76,3.92,0.79
2019-01-04,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,0.0,0.0,4.42,0.0,...,4.31,6.21,4.14,4.51,0.00,2.30,0.78,3.71,3.65,0.00
2019-01-05,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,0.0,0.0,4.86,0.0,...,4.49,6.06,3.51,4.92,0.68,2.19,0.00,1.66,2.65,0.00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-15,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,42.0,0.0,7.79,0.0,...,7.42,8.51,5.87,6.95,0.94,2.50,0.94,4.42,5.36,0.00
2022-05-16,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,57.0,0.0,7.90,0.0,...,7.87,8.75,4.83,5.16,0.62,3.06,1.33,3.33,5.25,0.00
2022-05-17,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,44.0,0.0,6.82,0.0,...,6.37,10.10,4.78,5.71,0.96,2.99,1.52,4.21,4.60,0.00


In [888]:
symptom_class = {}
if weight_type == 'ff100':
    classes = ["Loss of the sense of smell", "Joint ache", "Altered consciousness", "Cough", "Diarrhea", "Fatigue", "Fever", "Headache", 
           "Muscle ache", "Nausea", "Nose bleed", "Runny nose", "Shortness of breath", "Rash", "Sore throat", "Vomiting", ] 

elif weight_type == 'ons':
    # classes = ["Abdominal pain",  "Loss of taste",  "Loss of smell",  "Fatigue weakness",  "Headache", "Cough", "Diarrhea", 
    #             "Loss of taste", "Fatigue weakness", "Fever", "Headache", "Fever", "Muscle ache myalgia", 
    #             "Nausea vomiting", "Runny nose", "Shortness of breath", "Shortness of breath", "Sore throat", "Sore throat", 
    #              "Nausea vomiting", ] 
#     classes = ["Abdominal pain",  "Loss of taste",  "Loss of smell",  "Fatigue weakness",  "Headache", "Cough", "Diarrhea", 
#                 "Loss of taste", "Fatigue weakness", "Fever", "Headache", "Fever", "Muscle ache myalgia", 
#                 "Nausea vomiting", "Runny nose", "Shortness of breath", "Shortness of breath", "Sore throat", "Sore throat", 
#                  "Nausea vomiting", "Fatigue weakness", ] 
#     classes = ["Loss of taste", "Loss of smell", "Cough", "Diarrhea", "Fatigue weakness", "Fever", "Headache", "Muscle ache myalgia", 
#                "Nausea vomiting", "Runny nose", "Shortness of breath", "Sore throat", "Nausea vomiting",  ] 
#     classes = ["Loss of taste", "Loss of smell", "Cough", "Diarrhea", "Fatigue weakness", "Fever", "Headache", "Muscle ache myalgia", 
#                "Nausea vomiting", "Shortness of breath", "Sore throat", "Nausea vomiting",  ] 
    classes = ["Abdominal pain",  "Loss of taste", "Loss of smell", "Cough", "Diarrhea", "Fatigue weakness", "Fever", "Headache", 
               "Muscle ache myalgia", "Nausea vomiting", "Shortness of breath", "Sore throat", "Nausea vomiting",  "Fatigue weakness", ] 
#     classes = ["Loss of taste", "Loss of smell", "Cough", "Diarrhea", "Fatigue weakness", "Fever", "Headache", "Muscle ache myalgia", 
#                "Nausea vomiting", "Shortness of breath", "Sore throat",  ] 
for s, c in zip(symptoms, classes):
    symptom_class[s] = c
symptom_class 

{'symptom:Abdominal pain': 'Abdominal pain',
 'symptom:Ageusia': 'Loss of taste',
 'symptom:Anosmia': 'Loss of smell',
 'symptom:Cough': 'Cough',
 'symptom:Diarrhea': 'Diarrhea',
 'symptom:Fatigue': 'Fatigue weakness',
 'symptom:Fever': 'Fever',
 'symptom:Headache': 'Headache',
 'symptom:Myalgia': 'Muscle ache myalgia',
 'symptom:Nausea': 'Nausea vomiting',
 'symptom:Shortness of breath': 'Shortness of breath',
 'symptom:Sore throat': 'Sore throat',
 'symptom:Vomiting': 'Nausea vomiting',
 'symptom:Weakness': 'Fatigue weakness'}

In [889]:
for key in dfs.keys():
    df_copy = dfs[key].copy()
    column = dfs[key].columns.get_loc(symptoms[0])
    df = dfs[key].iloc[:, :column]
    
    class_set = list(set(classes))
    for c in class_set:
        df = pd.concat([df, pd.Series(np.zeros(len(df)), index=df.index, name=c)], axis=1)

    for c in class_set:
        for s in symptoms:
            if symptom_class[s] == c:
                df[c] = df[c] + df_copy[s]
    dfs[key] = df
dfs['Aberdeen City']

,isoCode,areaCode,parentCode,isoName,googleName,nhsName,newCasesBySpecimenDate,newDeaths28DaysByDeathDate,Diarrhea,Fatigue weakness,Fever,Loss of taste,Cough,Abdominal pain,Nausea vomiting,Shortness of breath,Muscle ache myalgia,Loss of smell,Sore throat,Headache
date,,,,,,,,,,,,,,,,,,,,
2019-01-01,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,0.0,0.0,4.83,4.03,3.47,0.0,8.37,5.14,5.49,1.01,0.00,0.0,2.71,4.31
2019-01-02,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,0.0,0.0,3.80,6.77,3.37,0.0,7.96,5.02,5.42,0.77,0.00,0.0,3.00,3.77
2019-01-03,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,0.0,0.0,4.54,7.11,4.12,0.0,9.05,5.70,5.92,0.00,0.59,0.0,2.76,4.43
2019-01-04,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,0.0,0.0,4.31,6.21,4.14,0.0,9.34,4.42,5.95,0.78,0.00,0.0,3.71,4.51
2019-01-05,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,0.0,0.0,4.49,6.06,3.51,0.0,7.97,4.86,4.84,0.00,0.68,0.0,1.66,4.92
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-15,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,42.0,0.0,7.42,8.51,5.87,0.0,6.95,7.79,7.86,0.94,0.94,0.0,4.42,6.95
2022-05-16,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,57.0,0.0,7.87,8.75,4.83,0.0,7.19,7.90,8.31,1.33,0.62,0.0,3.33,5.16
2022-05-17,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,44.0,0.0,6.37,10.10,4.78,0.0,6.67,6.82,7.59,1.52,0.96,0.0,4.21,5.71


In [890]:
column = dfs['Aberdeen City'].columns.get_loc(metrics[0])
index = dfs['Aberdeen City'].index
columns = dfs['Aberdeen City'].columns

for key in dfs.keys():
    dfs[key] = pd.concat([dfs[key].iloc[:, :column], pd.DataFrame(dfs[key].iloc[:, column:].rolling(7).mean(), index=index, columns=columns[column:])], axis=1)
    dfs[key] = dfs[key].dropna(how='any')
dfs['Aberdeen City']

,isoCode,areaCode,parentCode,isoName,googleName,nhsName,newCasesBySpecimenDate,newDeaths28DaysByDeathDate,Diarrhea,Fatigue weakness,Fever,Loss of taste,Cough,Abdominal pain,Nausea vomiting,Shortness of breath,Muscle ache myalgia,Loss of smell,Sore throat,Headache
date,,,,,,,,,,,,,,,,,,,,
2019-01-07,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,0.000000,0.0,4.405714,6.288571,3.740000,0.0,8.418571,5.032857,5.482857,0.598571,0.282857,0.0,2.817143,4.367143
2019-01-08,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,0.000000,0.0,4.371429,6.805714,3.787143,0.0,8.167143,4.951429,5.461429,0.454286,0.458571,0.0,2.852857,4.254286
2019-01-09,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,0.000000,0.0,4.475714,6.644286,3.838571,0.0,8.202857,4.951429,5.448571,0.425714,0.458571,0.0,2.807143,4.311429
2019-01-10,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,0.000000,0.0,4.430000,6.462857,3.911429,0.0,7.831429,4.690000,5.351429,0.560000,0.454286,0.0,2.770000,4.321429
2019-01-11,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,0.000000,0.0,4.388571,6.344286,3.864286,0.0,7.510000,4.797143,5.194286,0.594286,0.532857,0.0,2.585714,4.288571
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-15,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,43.714286,0.0,7.001429,8.942857,5.445714,0.0,7.050000,8.054286,7.527143,1.241429,0.614286,0.0,3.812857,5.744286
2022-05-16,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,43.285714,0.0,7.131429,8.681429,5.355714,0.0,7.020000,8.068571,7.678571,1.195714,0.578571,0.0,3.760000,5.832857
2022-05-17,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,43.142857,0.0,7.061429,8.790000,5.348571,0.0,6.895714,7.708571,7.785714,1.185714,0.622857,0.0,3.760000,5.870000


In [892]:
weights = pd.read_excel('data/symptoms/datadownload.xlsx', sheet_name=1, index_col=None) 
weights

,Abdominal pain,Cough,Diarrhea,Fatigue weakness,Fever,Headache,Loss of smell,Loss of taste,Muscle ache myalgia,Nausea vomiting,Shortness of breath,Sore throat,No reported symptoms
0,7.315294,41.814118,6.815294,37.253529,24.429412,37.633529,18.004118,18.753529,25.725294,9.484706,14.321765,32.065294,38.638235


In [893]:
symptom_weights={}
column = df.columns.get_loc(class_set[0])

if weight_type == 'ff100':
    weights = {'Fatigue':0.709, 'Fever':0.601, 'Loss of the sense of smell':0.291, 'Nausea':0.236, 'Shortness of breath':0.404,
                   'Diarrhea':0.276, 'Muscle ache':0.509, 'Runny nose':0.325, 'Vomiting':0.087,  'Rash': 0.052,
                   'Cough':0.777, 'Nose bleed':0.06, 'Headache':0.567,  'Sore throat':0.386, 'Joint ache':0.339, 'Altered consciousness':0.068,}
    for s in df.columns[column:]:
        symptom_weights[s] = weights[s] 
    
elif weight_type == 'ons':
    weights = pd.read_excel('data/symptoms/datadownload.xlsx', sheet_name=1, index_col=None).drop(['No reported symptoms'], axis=1) 
    weights = pd.concat([weights, pd.Series([.325*100], index=weights.index, name='Runny nose')], axis=1)
    for s in df.columns[column:]:
        if s == 'Runny nose':
            symptom_weights[s] = .325*100
        elif s=='Joint ache':
            symptom_weights[s] = .339*100
        else:
            symptom_weights[s] = weights[s].values[0]

symptom_weights

{'Diarrhea': 6.815294117647059,
 'Fatigue weakness': 37.25352941176472,
 'Fever': 24.42941176470588,
 'Loss of taste': 18.753529411764706,
 'Cough': 41.81411764705882,
 'Abdominal pain': 7.315294117647059,
 'Nausea vomiting': 9.48470588235294,
 'Shortness of breath': 14.321764705882355,
 'Muscle ache myalgia': 25.725294117647056,
 'Loss of smell': 18.004117647058823,
 'Sore throat': 32.065294117647056,
 'Headache': 37.633529411764705}

In [894]:
index = dfs['Aberdeen City'].index
column = dfs['Aberdeen City'].columns.get_loc('newDeaths28DaysByDeathDate')+1
for key in dfs.keys():
    score =pd.DataFrame([np.average(row[column:], weights=list(symptom_weights.values())) for row in dfs[key].values], 
                        index=index, columns=['search_score'])
    dfs[key] = pd.concat([dfs[key],score], axis=1)
dfs['Aberdeen City']

,isoCode,areaCode,parentCode,isoName,googleName,nhsName,newCasesBySpecimenDate,newDeaths28DaysByDeathDate,Diarrhea,Fatigue weakness,...,Loss of taste,Cough,Abdominal pain,Nausea vomiting,Shortness of breath,Muscle ache myalgia,Loss of smell,Sore throat,Headache,search_score
date,,,,,,,,,,,,,,,,,,,,,
2019-01-07,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,0.000000,0.0,4.405714,6.288571,...,0.0,8.418571,5.032857,5.482857,0.598571,0.282857,0.0,2.817143,4.367143,3.899743
2019-01-08,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,0.000000,0.0,4.371429,6.805714,...,0.0,8.167143,4.951429,5.461429,0.454286,0.458571,0.0,2.852857,4.254286,3.929797
2019-01-09,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,0.000000,0.0,4.475714,6.644286,...,0.0,8.202857,4.951429,5.448571,0.425714,0.458571,0.0,2.807143,4.311429,3.921026
2019-01-10,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,0.000000,0.0,4.430000,6.462857,...,0.0,7.831429,4.690000,5.351429,0.560000,0.454286,0.0,2.770000,4.321429,3.838220
2019-01-11,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,0.000000,0.0,4.388571,6.344286,...,0.0,7.510000,4.797143,5.194286,0.594286,0.532857,0.0,2.585714,4.288571,3.748198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-15,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,43.714286,0.0,7.001429,8.942857,...,0.0,7.050000,8.054286,7.527143,1.241429,0.614286,0.0,3.812857,5.744286,4.791489
2022-05-16,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,43.285714,0.0,7.131429,8.681429,...,0.0,7.020000,8.068571,7.678571,1.195714,0.578571,0.0,3.760000,5.832857,4.752381
2022-05-17,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,43.142857,0.0,7.061429,8.790000,...,0.0,6.895714,7.708571,7.785714,1.185714,0.622857,0.0,3.760000,5.870000,4.748626


In [895]:
population_stats = pd.read_excel('data/GB/population/ukpopestimatesmid2020on2021geography.xls', sheet_name=6, header=7) 
population_region = set(population_stats['Name'])
populations = {}
for key in dfs.keys():
    if key == 'Belfast City':
        populations[key] = float(population_stats[population_stats['Name']=='Belfast']['All ages'].values[0])
    elif key == 'Derry and Strabane':
        populations[key] = float(population_stats[population_stats['Name']=='Derry City and Strabane']['All ages'].values[0])
    elif key == 'Durham, County':
        populations[key] = float(population_stats[population_stats['Name']=='County Durham']['All ages'].values[0])
    elif key == 'Edinburgh, City of':
        populations[key] = float(population_stats[population_stats['Name']=='City of Edinburgh']['All ages'].values[0])
    elif key == 'Eilean Siar':
        populations[key] = float(population_stats[population_stats['Name']=='Na h-Eileanan Siar']['All ages'].values[0])
    elif key == 'Herefordshire':
        populations[key] = float(population_stats[population_stats['Name']=='Herefordshire, County of']['All ages'].values[0])
    elif key == 'Kingston upon Hull':
        populations[key] = float(population_stats[population_stats['Name']=='Kingston upon Hull, City of']['All ages'].values[0])
    elif key == 'Mid-Ulster':
        populations[key] = float(population_stats[population_stats['Name']=='Mid Ulster']['All ages'].values[0])
    elif key == 'Rhondda Cynon Taff':
        populations[key] = float(population_stats[population_stats['Name']=='Rhondda Cynon Taf']['All ages'].values[0])
    elif key == 'Northamptonshire':
        populations[key] = 757000
    else:
        populations[key] = float(population_stats[population_stats['Name']==key]['All ages'].values[0])

In [897]:
idx = dfs['Aberdeen City'].columns.get_loc('nhsName')+1
for key in dfs.keys():
    dfs[key].insert(idx, 'population', populations[key])
dfs['Aberdeen City']

,isoCode,areaCode,parentCode,isoName,googleName,nhsName,population,newCasesBySpecimenDate,newDeaths28DaysByDeathDate,Diarrhea,...,Loss of taste,Cough,Abdominal pain,Nausea vomiting,Shortness of breath,Muscle ache myalgia,Loss of smell,Sore throat,Headache,search_score
date,,,,,,,,,,,,,,,,,,,,,
2019-01-07,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,229060.0,0.000000,0.0,4.405714,...,0.0,8.418571,5.032857,5.482857,0.598571,0.282857,0.0,2.817143,4.367143,3.899743
2019-01-08,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,229060.0,0.000000,0.0,4.371429,...,0.0,8.167143,4.951429,5.461429,0.454286,0.458571,0.0,2.852857,4.254286,3.929797
2019-01-09,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,229060.0,0.000000,0.0,4.475714,...,0.0,8.202857,4.951429,5.448571,0.425714,0.458571,0.0,2.807143,4.311429,3.921026
2019-01-10,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,229060.0,0.000000,0.0,4.430000,...,0.0,7.831429,4.690000,5.351429,0.560000,0.454286,0.0,2.770000,4.321429,3.838220
2019-01-11,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,229060.0,0.000000,0.0,4.388571,...,0.0,7.510000,4.797143,5.194286,0.594286,0.532857,0.0,2.585714,4.288571,3.748198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-15,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,229060.0,43.714286,0.0,7.001429,...,0.0,7.050000,8.054286,7.527143,1.241429,0.614286,0.0,3.812857,5.744286,4.791489
2022-05-16,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,229060.0,43.285714,0.0,7.131429,...,0.0,7.020000,8.068571,7.678571,1.195714,0.578571,0.0,3.760000,5.832857,4.752381
2022-05-17,GB-ABE,S12000033,GB-SCT,Aberdeen City,Aberdeen City,Aberdeen City,229060.0,43.142857,0.0,7.061429,...,0.0,6.895714,7.708571,7.785714,1.185714,0.622857,0.0,3.760000,5.870000,4.748626


In [941]:
def standardisation(df, mean=None, std=None, given=False):
    if given==False:
        mean = df.mean()
        std = df.std()
    return (df-mean)/std, mean, std

def standard_inverse(df, mean, std):
    return df*std+mean

In [942]:
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.tsa.ar_model import AutoReg
from sklearn.preprocessing import StandardScaler

def compute_gamma(series, news, lags=2, n=56, standardise=True):
    series = pd.DataFrame(series.values, index=series.index, columns=['search_score'])
    y = series.iloc[lags:]
    x = pd.DataFrame(index=y.index, columns=[])
    for i in range(1, lags+1):
        x['ar_'+str(i)] = series.shift(i)[lags:]
    for i in range(0, lags+1):
        x['news_'+str(i)] = news.shift(i)[lags:]
    
    error1 = []
    error2 = []
    predictions_ar = []
    predictions_news = []
    gammas = {}
    gammas_process = {}
    for i in range(len(y)-n-1):
        if standardise==True:
            y_train, y_mean, y_std = standardisation(y.iloc[i:n+i])
            y_test, _, _ = standardisation(y.iloc[1+i:n+1+i], y_mean, y_std, given=True)
            x_train, x_mean, x_std = standardisation(x.iloc[i:n+i, :])
            x_test, _, _ = standardisation(x.iloc[1+i:n+1+i, :], x_mean, x_std, given=True)
            x_train = sm.add_constant(x_train)
            x_test = sm.add_constant(x_test)
        else:
            y_train = y.iloc[i:n+i]
            y_test = y.iloc[1+i:n+1+i]
            x_train = x.iloc[i:n+i, :]
            x_test = x.iloc[1+i:n+1+i, :]
            x_train = sm.add_constant(x_train)
            x_test = sm.add_constant(x_test)
        
        result_ar = sm.OLS(y_train, x_train.iloc[:, :lags+1]).fit()
        result_news = sm.OLS(y_train, x_train).fit()
        pred_ar = result_ar.predict(x_test.iloc[:, :lags+1])
        pred_news = result_news.predict(x_test)
        predictions_ar.append(pred_ar)
        predictions_news.append(pred_news)
    
        e1 = np.abs(y_test['search_score'].values - pred_ar).mean()
        e2 = np.abs(y_test['search_score'].values - pred_news).mean()     
        error1.append(e1)
        error2.append(e2)
        ratio = e2/e1
        gammas[y.index[n+i+1]] = ratio
        if ratio < 1:
            gammas_process[y.index[n+i+1]] = ratio
        else:
            gammas_process[y.index[n+i+1]] = 1.
        
    return gammas_process, gammas, error1, error2, predictions_ar, predictions_news

In [943]:
news = pd.read_csv('data/GB/news_UK.csv')
news['date'] = pd.to_datetime(news['date'].astype(str))
news = news.set_index('date')
news = news.rename(columns={'covid-news-ratio-UK': 'news'})
for key in dfs.keys():
    dfs[key] = pd.merge(dfs[key], news, on='date')

date
2019-01-07    3.899743
2019-01-08    3.929797
2019-01-09    3.921026
2019-01-10    3.838220
2019-01-11    3.748198
                ...   
2022-05-15    4.791489
2022-05-16    4.752381
2022-05-17    4.748626
2022-05-18    4.708939
2022-05-19    4.614077
Name: search_score, Length: 1229, dtype: float64

In [946]:
for key in dfs.keys():
    gammas = compute_gamma(dfs[key]['search_score'], dfs[key]['news'], lags=2, n=120, standardise=False)[0]
    gammas = pd.DataFrame(gammas.values(), index=gammas.keys(), columns=['gamma'])
    gammas.index.name = 'date'
    gammas = gammas.rolling(7).mean().dropna()
    dfs[key] = pd.merge(dfs[key], gammas, on='date').dropna(how='any')
    dfs[key]['search_score_adj'] =  dfs[key]['search_score']*dfs[key]['gamma']
    dfs[key] = dfs[key].drop(columns=['news', 'gamma'])

In [898]:
def correlation(df, metric='newCasesBySpecimenDate', data_range=None, shift=0):
    if data_range is not None:
        start = data_range[0]
        end = data_range[1]
    else:
        start = df.index[0]
        end = df.index[-1]
    
    correl = {}
    for i in range(df.columns.get_loc(class_set[0]), len(df.columns)):
        clinical = df[metric].shift(shift).loc[start:end].dropna()
#         if np.all(list(df[df.columns[i]].values)==0):
#             correl[df.columns[i]] = 0.0
#         else:
        correl[df.columns[i]] = np.corrcoef(clinical, df[df.columns[i]].loc[clinical.index[0]:clinical.index[-1]])[0][1]

    
    return correl

In [899]:
def correl_series(df, metric='newCasesBySpecimenDate', data_range=None, shift_range=(-30, 30)):
    days = np.arange(shift_range[0], shift_range[1]+1)
    for i, d in enumerate(days):
        if i == 0:
            correl = correlation(df, metric=metric, data_range=data_range, shift=d)
            matrix = pd.DataFrame(np.round(list(correl.values()),decimals=3), index=correl.keys(), columns=[d])
        else:
            correl = correlation(df, metric=metric, data_range=data_range, shift=d)
            tmp = pd.DataFrame(np.round(list(correl.values()),decimals=3), index=correl.keys(), columns=[d])
            matrix = pd.concat([matrix, tmp], axis=1)
    
    return matrix

In [900]:
def dipict_correl_series(df, metric='newCasesBySpecimenDate', start='2020-01-01', end='2022-05-18', 
                         shift_range=(-30, 30), score='search_score'):
    plt.figure(figsize=(12, 9)) 
    matrix = correl_series(df, metric=metric, data_range=(start, end), shift_range=shift_range)
    print('Correlation of {} from {} to {}:'.format(metric, start, end))
    print('  ', 0, ':', matrix.loc[score, 0])
    print('  ', list(matrix.loc[score]).index(matrix.loc[score].max())-(shift_range[0]*-1), ':', matrix.loc[score].max())
    plt.plot(matrix.iloc[:-2].T)
    plt.plot(matrix.iloc[-2].T, color=(0.1, 0.2, 0.5))
    plt.plot(matrix.iloc[-1].T, color='black', lw=3)
    plt.legend(matrix.index)
    plt.ylabel('Correlation')
    plt.xlabel('Number of days the clinical data shifted')
    plt.title('Correlation of search score and {} by shifted days: {} to {}'.format(metric, start, end))
#     plt.savefig('images/country/correl_series/{}/{}_{}_{}.pdf'.format(weight_type, metric, start, end))
    return list(matrix.loc[score]).index(matrix.loc[score].max())-(shift_range[0]*-1)

In [901]:
def depict_series(df, metric='newCasesBySpecimenDate', start='2020-02-13', end='2022-05-18', shift=None, score='search_score'):
    start_idx  = df.index.get_loc(start)
    end_idx = df.index.get_loc(end)
    fig = plt.figure(figsize=(12, 9)) 
    ax1 = fig.add_subplot(111)
    ln1 = ax1.plot(df.index[start_idx:end_idx], df[score].iloc[start_idx:end_idx], label='Search score')
    ax2 = ax1.twinx()
    ln2 = ax2.plot(df.index[start_idx:end_idx], df[metric].iloc[start_idx:end_idx], color='red', label=metric)
    if shift is not None:
        shifted = df[metric].shift(shift).iloc[start_idx:end_idx].dropna()
        start_shifted  = df.index.get_loc(shifted.index[0])
        end_shifted = df.index.get_loc(shifted.index[-1])+1
        ax2.plot(df.index[start_shifted:end_shifted], shifted, color='red', lw=1, ls='--', label=metric + '(shifted)')
    ax1.set_xlabel("Numbers of days from {}".format(start))
    ax1.set_ylabel("Search Score")
    ax2.set_ylabel(metric)
    plt.title('Search score and {}: {} to {}'.format(metric, start, end))
    h1, l1 = ax1.get_legend_handles_labels()
    h2, l2 = ax2.get_legend_handles_labels()
    ax1.legend(h1+h2, l1+l2)
#     plt.savefig('images/country/time_series/{}/{}/{}_{}_{}.pdf'.format(weight_type, score, metric, start, end))

In [906]:
area = ['Barnet', 'Birmingham', 'Bournemouth, Christchurch and Poole', 'Bradford', 'Bromley', 'Cambridgeshire', 'Derbyshire', 'Devon', 'Ealing', 'Essex', 'Glasgow City', 'Gloucestershire', 'Hampshire', 'Hertfordshire', 'Kent', 'Kirklees', 'Lambeth', 'Lancashire', 'Leeds', 'Leicestershire', 'Lincolnshire', 'Liverpool', 'Manchester', 'Newham', 'Norfolk', 'North Yorkshire', 'Northamptonshire', 'Nottinghamshire', 'Oxfordshire', 'Sheffield', 'Southwark', 'Suffolk', 'Surrey', 'Tower Hamlets', 'Wandsworth', 'Warwickshire', 'West Sussex', 'Westminster']
dfs_tmp = {}
for key in dfs.keys():
    if key in area:
        dfs_tmp[key] = dfs[key]

In [1013]:
metrics = ['newCasesBySpecimenDate', 'newDeaths28DaysByDeathDate']
p = ['2020-02-13', '2022-05-19']
p1 = ['2020-02-13', '2020-07-15']
p2 = ['2020-07-16', '2021-03-31']
p3 = ['2020-02-13', '2021-03-31']
p4 = ['2020-07-16', '2022-05-19']
p5 = ['2021-04-01', '2022-05-19']
periods = [p, p1, p2, p3, p4, p5]
scores = ['search_score', 'search_score_adj']
shift_range=(-40, 10)

shifted_correl_tables = {}
for m in metrics:
    for p in periods:
        for s in scores:
            shifted_correl = pd.DataFrame(index=np.arange(shift_range[0], shift_range[1]+1,1), columns=dfs.keys())
            for key in dfs.keys():
                df = dfs[key]
                correls = []
                for shift in np.arange(shift_range[0], shift_range[1]+1,1):
                    clinical = df[m].shift(shift).loc[p[0]:p[1]].dropna()
                    search = df[s].loc[clinical.index[0]:clinical.index[-1]]
                    correls.append(np.corrcoef(clinical, search)[0][1])
                shifted_correl[key] = pd.Series(correls, index=np.arange(shift_range[0], shift_range[1]+1,1))
            shifted_correl_tables[m+p[0]+p[1]+s] = shifted_correl

In [1019]:
for key in shifted_correl_tables.keys():
    print('{} : mean shifted days {} : mean max correl {} : std {}'.format(key, round(shifted_correl_tables[key].idxmax().mean(),1), 
         round(shifted_correl_tables[key].max().mean(),3), round(shifted_correl_tables[key].max().std(),3)))

newCasesBySpecimenDate2020-02-132022-05-19search_score : mean shifted days -7.0 : mean max correl 0.244 : std 0.052
newCasesBySpecimenDate2020-02-132022-05-19search_score_adj : mean shifted days -8.5 : mean max correl 0.268 : std 0.053
newCasesBySpecimenDate2020-02-132020-07-15search_score : mean shifted days -30.8 : mean max correl 0.805 : std 0.163
newCasesBySpecimenDate2020-02-132020-07-15search_score_adj : mean shifted days -31.2 : mean max correl 0.804 : std 0.163
newCasesBySpecimenDate2020-07-162021-03-31search_score : mean shifted days -17.0 : mean max correl 0.372 : std 0.185
newCasesBySpecimenDate2020-07-162021-03-31search_score_adj : mean shifted days -17.2 : mean max correl 0.393 : std 0.181
newCasesBySpecimenDate2020-02-132021-03-31search_score : mean shifted days -24.5 : mean max correl 0.008 : std 0.06
newCasesBySpecimenDate2020-02-132021-03-31search_score_adj : mean shifted days -24.3 : mean max correl 0.026 : std 0.061
newCasesBySpecimenDate2020-07-162022-05-19search_sc

In [1018]:
for key in shifted_correl_tables.keys():
    print('{} : 0 : correl {}'.format(key, round(shifted_correl_tables[key].loc[0].mean(),3)))

newCasesBySpecimenDate2020-02-132022-05-19search_score : 0 : correl 0.235
newCasesBySpecimenDate2020-02-132022-05-19search_score_adj : 0 : correl 0.258
newCasesBySpecimenDate2020-02-132020-07-15search_score : 0 : correl -0.079
newCasesBySpecimenDate2020-02-132020-07-15search_score_adj : 0 : correl -0.101
newCasesBySpecimenDate2020-07-162021-03-31search_score : 0 : correl 0.254
newCasesBySpecimenDate2020-07-162021-03-31search_score_adj : 0 : correl 0.271
newCasesBySpecimenDate2020-02-132021-03-31search_score : 0 : correl -0.075
newCasesBySpecimenDate2020-02-132021-03-31search_score_adj : 0 : correl -0.062
newCasesBySpecimenDate2020-07-162022-05-19search_score : 0 : correl 0.456
newCasesBySpecimenDate2020-07-162022-05-19search_score_adj : 0 : correl 0.467
newCasesBySpecimenDate2021-04-012022-05-19search_score : 0 : correl 0.343
newCasesBySpecimenDate2021-04-012022-05-19search_score_adj : 0 : correl 0.359
newDeaths28DaysByDeathDate2020-02-132022-05-19search_score : 0 : correl -0.051
newDe